# Pre-processing
1. Read miniseed data
2. Update header variables
3. Write SAC files
4. Filter

In [ ]:
from obspy import UTCDateTime
import obspy
import os
from obspy.taup import TauPyModel
from distaz import DistAz

# Define functions

In [ ]:
def read_evt(evtf):
    """
    read event information
    """
    with open(evtf, 'r') as f:
        lines = f.readlines()[1:]
    events = []
    dnames = []
    for line in lines:
        line = line.strip()
        temp = line.split(',')
        t = UTCDateTime(temp[0])
        lat, lon = float(temp[1]), float(temp[2])
        depth = float(temp[3])
        dpu = temp[4]
        mag = float(temp[5])
        magt = temp[6]
        events.append([t, lat, lon, depth, dpu, mag, magt])
        dnames.append("%04d%02d%02d%02d%02d%02d%s" % (t.year, t.month, t.day, t.hour, t.minute, t.second, str(t.microsecond)[:3]))
    return dnames, events


def read_sta(staf):
    """
    read station information
    """
    with open(staf, 'r') as f:
        lines = f.readlines()
    stations, name = [], []
    for line in lines:
        line=line.strip()
        #if (not line) or line[0] == '#':
        #    continue
        temp = line.split(' ')
        name.append(temp[0])
        lat, lon, stel = float(temp[1]), float(temp[2]), float(temp[3])
        stations.append([lat, lon, stel])
    return name, stations


def mseed2sac(data_root, data_out, dnames, events, fnames, stations):
    model = TauPyModel(model="iasp91")
    for evt in zip(dnames, events):
        for sta in zip(fnames, stations):
            fpath = os.path.join(data_root, evt[0], sta[0] + '.mseed')
            try:
                st = obspy.read(fpath)
            except FileNotFoundError:
                print("Missing File: %s" % fpath)
                continue
            st.merge(method=1, fill_value=0, interpolation_samples=-1)
            # convert to sac
            st.write('temp.sac', format='SAC')
            st = obspy.read('temp.sac')
            os.remove('./temp.sac')

            st[0].stats.sac.stla = sta[1][0]
            st[0].stats.sac.stlo = sta[1][1]
            st[0].stats.sac.stel = sta[1][2]
            st[0].stats.sac.evla = evt[1][1]
            st[0].stats.sac.evlo = evt[1][2]
            st[0].stats.sac.evdp = evt[1][3]
            st[0].stats.sac.mag = evt[1][5]
            st[0].stats.sac.o = evt[1][0] - st[0].stats.starttime
            st[0].stats.sac.lcalda = 1

            da = DistAz(sta[1][0], sta[1][1], evt[1][1], evt[1][2])
            gcarc = da.getDelta()
            arrivals = model.get_travel_times(source_depth_in_km=evt[1][3],
                                  distance_in_degree=gcarc, phase_list=['ttall'])
            st[0].stats.sac.t0 = arrivals[0].time + st[0].stats.sac.o

            outd = os.path.join(data_out, evt[0])
            if not os.path.exists(outd):
                os.makedirs(outd)
            st.write(os.path.join(outd, sta[0]+'.SAC'))


def do_filter(data_dir, freq_low, freq_high, out_dir):
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    for e in os.listdir(data_dir):
        inpath = os.path.join(data_dir, e)
        outpath = os.path.join(out_dir, e)
        if not os.path.exists(outpath):
            os.mkdir(outpath)
        for sac in os.listdir(inpath):
            insac = os.path.join(inpath, sac)
            outsac = os.path.join(outpath, sac)
            st = obspy.read(insac)
            st.filter('bandpass', freqmin=freq_low, freqmax=freq_high)
            st[0].write(outsac, format='SAC')


def do_rotate(data_dir):
    pass

# Call functions

In [ ]:
data_root = './miniseed/'
data_sac = './sac/raw/'
evt_lst = './events.csv'
sta_lst = './station.dat'
if not os.path.exists(data_sac):
    os.makedirs(data_sac)
dnames, events = read_evt(evt_lst)
fnames, stations = read_sta(sta_lst)
mseed2sac(data_root, data_sac, dnames, events, fnames, stations)
do_filter(data_sac, 0.1, 10, "./sac/filt_0.1_10Hz")

# Benchmark
check the header variables

In [ ]:
ref = './seismograms/'
flog = open('bench.log', 'w')
def bench_header(stats):
    return [stats.starttime+stats.sac.o-stats.sac.b, stats.sac.stla, stats.sac.stlo, stats.sac.evla, stats.sac.evlo, stats.sac.evdp, stats.sac.mag, stats.sac.stel]
def diff_header(s0, s1):
    result = []
    for h0, h1 in zip(s0, s1):
        if h0 - h1 < 0.01:
            result.append('OK')
        else:
            result.append('XX')
    return result
for d in os.listdir(ref):
    d2 = os.path.join(ref, d)
    for fname in os.listdir(d2):
        pref = os.path.join(d2, fname)
        psac = os.path.join(data_sac, d, fname)
        tr0 = obspy.read(pref)[0]
        tr1 = obspy.read(psac)[0]
        s0 = tr0.stats
        s1 = tr1.stats
        flog.write("---------------------------------------------\n")
        cur = ""
        cur += "\t".join(["sourcetime", "stla", "stlo", "evla", "evlo", "evdp", "mag", "stel", "\n"])
        cur += "\t".join(map(lambda x: str(x), bench_header(s0) + ['\n']))
        cur += "\t".join(map(lambda x: str(x), bench_header(s1) + ['\n']))
        cur += "\t".join(map(lambda x: str(x), diff_header(bench_header(s0), bench_header(s1)) + ['\n']))
        flog.write(cur)
        flog.write("---------------------------------------------\n")
flog.close()